In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

from tqdm import tqdm

# model
import lightgbm as lgb

SEED=42
LABEL=None

In [0]:
path = '/content/drive/My Drive/bigcontest2019/'
os.chdir(path)

In [0]:
train_activity = pd.read_csv('data/train/train_activity.csv')
# train_combat = pd.read_csv('data/train/train_combat.csv')
train_payment = pd.read_csv('data/train/train_payment.csv')
# train_pledge = pd.read_csv('data/train/train_pledge.csv')
# train_trade = pd.read_csv('data/train/train_trade.csv')

test1_activity = pd.read_csv('data/test/test1_activity.csv')
# test1_combat = pd.read_csv('data/test/test1_combat.csv')
test1_payment = pd.read_csv('data/test/test1_payment.csv')
# test1_pledge = pd.read_csv('data/test/test1_pledge.csv')
# test1_trade = pd.read_csv('data/test/test1_trade.csv')

test2_activity = pd.read_csv(path + 'data/test/test2_activity.csv')
# test2_combat = pd.read_csv(path + 'data/test/test2_combat.csv')
test2_payment = pd.read_csv(path + 'data/test/test2_payment.csv')
# test2_pledge = pd.read_csv(path + 'data/test/test2_pledge.csv')
# test2_trade = pd.read_csv(path + 'data/test/test2_trade.csv')

train_label = pd.read_csv('data/train/train_label.csv')

In [0]:
class data_transform(object):
    def __init__(self, data):
        self.data = data
        
    def create_week(self):
        self.data['week'] = (self.data['day']-1)//7 + 1
        
        return self.data
    
#     def ():
#         return 
    
    def groupby_week(self):
        temp_df = self.data
        
        activity_agg = {'day':'nunique', 
               #'char_id':'nunique',
               #'server':'nunique', 
               'playtime':'sum', 
               'npc_kill':'sum', 
               'solo_exp':'sum',
               'party_exp':'sum', 
               'quest_exp':'sum',
               'rich_monster':'sum', 
               'death':'sum', 
               'revive':'sum',
               'exp_recovery':'sum',
               'fishing':'sum',
               'private_shop':'sum',
               'game_money_change':'sum',
               'enchant_count':'sum'}
        
        temp_df = temp_df.groupby(['acc_id', 'week']).agg(activity_agg).reset_index()
        
        return temp_df
    
    
    def under_sampling(self):
        temp_df = self.data
        output_df = pd.DataFrame()
        for week in [1, 2, 3, 4]:
            _, under_sample = train_test_split(temp_df[(temp_df['week']==week) & (temp_df['survival_time']==64)], test_size=0.01, random_state=42, shuffle=True, 
                                               stratify=temp_df.loc[(temp_df['week']==week) & (temp_df['survival_time']==64), 'day'])
            output_df = pd.concat([output_df, under_sample]).reset_index(drop=True)
        
        return output_df
    
    def under_sampling2(self):
        temp_df = self.data
        _, under_sample = train_test_split(temp_df[(temp_df['survival_time']==64)], test_size=0.01, random_state=42, shuffle=True)
#                                            stratify=temp_df.loc[(temp_df['survival_time']==64), 'daynunique'])
        
        return under_sample

In [0]:
|

# train

## def function

In [0]:
def activity_merge(train_week, data, week=1, on='acc_id'):
    WEEK=week
    ON=on
    for method in ['nunique', 'count']:
        train_week = pd.merge(train_week, activity_proprocess(DATA, feature='server', method=method, week=WEEK), how='left', on=ON)

    for feature in ['playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing', 'private_shop', 'game_money_change', 'enchant_count']:
        for method in ['nunique', 'max', 'min', 'mean', 'median', 'skew', 'kurt', 'sum', 'std']:
            train_week = pd.merge(train_week, activity_proprocess(DATA, feature=feature, method=method, week=WEEK), how='left', on=ON)
    
    return train_week

In [0]:
def payment_merge(train_week, data, week=1, on='acc_id'):
    WEEK=week
    ON=on
    feature = 'amount_spent'
    
    for method in ['nunique', 'max', 'min', 'mean', 'median', 'skew', 'kurt', 'sum', 'std']:
        train_week = pd.merge(train_week, activity_proprocess(DATA, feature=feature, method=method, week=WEEK), how='left', on=ON)
    
    return train_week

## activity

In [0]:
transform = data_transform(train_activity)
transform.create_week()
train = transform.groupby_week()

## Payment

In [0]:
train2 = train_payment.groupby('acc_id').agg({'amount_spent':['sum', 'max', 'mean'], 'day':'nunique'}).reset_index()
train2.columns = [i+j for i,j in train2.columns.ravel()]
train2 = pd.merge(train2, train_label, how='left', on='acc_id').fillna(0)

In [0]:
# under sampling
transform = data_transform(train2)
under_train = transform.under_sampling2()

train2 = pd.concat([under_train, train2[train2['survival_time']!=64]]).reset_index(drop=True)

In [0]:
train_label['amount_spent_round'] = np.round(train_label['amount_spent'], 2)

In [0]:
SEED=42
stack_df = pd.DataFrame()
np.random.seed(SEED)

for as_round in train_label['amount_spent_round'].unique():
    temp_df = train_label[train_label['amount_spent_round']==as_round]
    try:
        temp_df = temp_df.loc[np.random.choice(temp_df.index, 3, replace=False)]
    except:
        pass
    stack_df = pd.concat([stack_df, temp_df])

try
1. under sampling
2. weight
3. total amount spent

weaknees
1. data loss
2. weak
3. unknown

# test

## activity

In [0]:
transform = data_transform(test1_activity)
transform.create_week()
test1 = transform.groupby_week()

In [0]:
transform = data_transform(test2_activity)
transform.create_week()
test2 = transform.groupby_week()

## payment

In [0]:
test12 = test1_payment.groupby('acc_id').agg({'amount_spent':['sum', 'max', 'mean'], 'day':'nunique'}).reset_index()
test12.columns = [i+j for i,j in test12.columns.ravel()]

# model

## survival_time

In [0]:
train = pd.merge(train, train_label[['acc_id', 'survival_time']], how='left', on='acc_id')

for week in range(1, 5):
    train.loc[train['week']==week, 'survival_time'] = np.minimum(64, train.loc[train['week']==week, 'survival_time'] + 7*(4-week))

In [0]:
# under sampling
transform = data_transform(train)
under_train = transform.under_sampling()

train = pd.concat([under_train, train[train['survival_time']!=64]]).reset_index(drop=True)

In [0]:
seed=42
LABEL='survival_time'
train_df, valid_df = train_test_split(train.fillna(0), test_size=0.2, random_state=seed, shuffle=True, stratify=train[LABEL])

In [0]:
feature_ = train.drop(columns=['acc_id', 'week', LABEL]).columns
params = {
    'objective':'multiclass',
    'num_class':64,
    "boosting": "gbdt",
    'learning_rate': 0.03,
    'subsample' : 0.6,
    'sumsample_freq':1,
    'colsample_bytree':0.221856,
    'max_depth': 16,
    'max_bin':255,
    "lambda_l1": 0.25,
    "lambda_l2": 1,
    'min_child_weight': 0.2,
    'min_child_samples': 20,
    'min_gain_to_split':0.02,
    'min_data_in_bin':3,
    'bin_construct_sample_cnt':5000,
    'cat_l2':10,
    'verbose':-1,
    'nthread':-1,
    'seed':seed
}

trn_label = train_df[LABEL] - 1
val_label = valid_df[LABEL] - 1

ttt = lgb.Dataset(train_df[feature_], label=trn_label)
vvv = lgb.Dataset(valid_df[feature_], label=val_label)

lgb_model = lgb.train(params, ttt, 5000, valid_sets = [ttt, vvv], early_stopping_rounds = 50, verbose_eval=100)

preds_st = lgb_model.predict(test1[test1['week']==4][feature_].fillna(0))
preds_st = np.argmax(preds_st, axis=1)+1

preds_st2 = lgb_model.predict(test2_week[feature_].fillna(0))
preds_st2 = np.argmax(preds_st2, axis=1)+1

## amount spent

In [0]:
seed=42
LABEL='amount_spent'
train_df, valid_df = train_test_split(train2.fillna(0), test_size=0.2, random_state=seed, shuffle=True)

In [370]:
rf = RandomForestRegressor(n_estimators=100).fit(stack_df[['survival_time']], stack_df['amount_spent'])

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
preds_spent = rf.predict(preds_st.reshape(-1, 1))

In [0]:
preds_spent2= rf.predict(preds_st2.reshape(-1, 1))

# submission

In [0]:
test1_pred = pd.concat([test1_week['acc_id'], pd.DataFrame(preds, columns=['survival_time']), pd.DataFrame(preds_spent*25, columns=['amount_spent'])], 1)
test1_pred.to_csv('/content/test1_predict.csv', index=False)

In [0]:
test2_pred = pd.concat([test2_week['acc_id'], pd.DataFrame(preds2, columns=['survival_time']), pd.DataFrame(preds2_spent*10, columns=['amount_spent'])], 1)
test2_pred.to_csv('/content/test2_predict.csv', index=False)

In [0]:
test1_pred = pd.concat([test1[test1['week']==4]['acc_id'].reset_index(drop=True), pd.DataFrame(preds_st, columns=['survival_time']), pd.DataFrame(preds_spent, columns=['amount_spent'])], 1)
test1_pred.to_csv('/content/test1_predict.csv', index=False)

In [0]:
test2_pred = pd.concat([test2[test2['week']==4]['acc_id'].reset_index(drop=True), pd.DataFrame(preds_st2, columns=['survival_time']), pd.DataFrame(preds_spent2, columns=['amount_spent'])], 1)
test2_pred.to_csv('/content/test2_predict.csv', index=False)